In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Download dependencies

In [2]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 8.3 MB/s 


In [3]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 9.9 MB/s 


In [4]:
!pip install wandb

     |████████████████████████████████| 1.7 MB 8.4 MB/s 
     |████████████████████████████████| 139 kB 58.7 MB/s 
     |████████████████████████████████| 97 kB 9.2 MB/s 
     |████████████████████████████████| 180 kB 69.2 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=bb358e296036c2b6e9828b5217d6602974835965ac9e025cf663d4811340e18e
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=04d30a30512794f54d7b97331dd88ac60a9ee6e7484cd059be36e2ed332da528
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


### Import libraries

In [35]:
import pandas as pd
import numpy as np
import re
import regex
from tqdm import tqdm

from sklearn import preprocessing, metrics
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

import tensorflow_addons as tfa
import tensorflow as tf

import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from string import punctuation
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout

import wandb
from wandb.keras import WandbCallback

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# print(torch.__version__)

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
# from torch import cuda
# device = 'cuda' if cuda.is_available() else 'cpu'

### Load data

In [9]:
f_path = '/content/drive/MyDrive/3.1/CZ4042 Neural Networks & Deep Learning/Project/emotion/dataset/crowdflower_text_emotion.csv'

In [10]:
df = pd.read_csv(f_path)

### Data preprocessing and analysis

In [11]:
df.head()

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...


In [12]:
# Check emotions labels

labels = df.sentiment.unique()
print(labels)

['empty' 'sadness' 'enthusiasm' 'neutral' 'worry' 'surprise' 'love' 'fun'
 'hate' 'happiness' 'boredom' 'relief' 'anger']


In [13]:
# Remove irrelevant columns
df = df.drop(columns=['tweet_id', 'author'])

In [14]:
# Check for null cells
df['sentiment'].isnull().values.any()


False

In [16]:
wordnet_lemmatizer = WordNetLemmatizer()

stop = stopwords.words('english')

for punct in punctuation:
    stop.append(punct)

def clean(x, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(x.lower())
    x = [regex.sub(u'\p{^Latin}', u'', w) for w in word_tokens if w.isalpha() and len(w) > 2]
    return " ".join(x)

df["content_cleaned"] = df.content.apply(lambda x : clean(x, stop)) 

In [17]:
# Check len of each content
df['content_count'] = df['content_cleaned'].str.split(" ").str.len()

In [18]:
# Check if any of the sequences are longer than 512
df[df.content_count > 512]

,sentiment,content,content_cleaned,content_count


In [19]:
df.head()

,sentiment,content,content_cleaned,content_count
0,empty,@tiffanylue i know i was listenin to bad habi...,tiffanylue know was listenin bad habit earlier...,12
1,sadness,Layin n bed with a headache ughhhh...waitin o...,layin bed with headache ughhhh waitin your call,8
2,sadness,Funeral ceremony...gloomy friday...,funeral ceremony gloomy friday,4
3,enthusiasm,wants to hang out with friends SOON!,wants hang out with friends soon,6
4,neutral,@dannycastillo We want to trade with someone w...,dannycastillo want trade with someone who has ...,12


### Drop unused columns

In [21]:
df = df.drop(columns=['content_count', 'content'])

### CNN

In [23]:
# Define max vocab size, max sequence length, and embedding dimension
MAX_NB_WORDS = 41277
MAX_SEQUENCE_LENGTH = 256
EMBEDDING_DIM = 128

# Initialize tokenizer and tokenize text
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['content_cleaned'].values)

# Get unique tokens/vocab size
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 41277 unique tokens.


In [24]:
# Convert to sequence
df.reset_index()
X = tokenizer.texts_to_sequences(df['content_cleaned'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

# Convert to numerical
Y = pd.get_dummies(df['sentiment']).values

print('Shape of data:', X.shape)
print('Shape of label:', Y.shape)

Shape of data: (40000, 256)
Shape of label: (40000, 13)


In [39]:
# Define hyperparameters

epochs = 50
batch_size = 8

In [40]:
# Define CNN filters
fil = [64, 128, 256]
# k_size = [3, 5, 7]

In [41]:
# Function to build model

def build_cnn(fil):
  model = Sequential()
  model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))

  model.add(Conv1D(fil, 3, padding='same', activation='relu'))
  model.add(MaxPooling1D())
  model.add(Flatten())
  model.add(Dropout(0.3))
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.3))
  model.add(Dense(13, activation='softmax'))

  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
  
  return model

In [42]:
# Run on GPU

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [43]:
# 3-Fold
cv = KFold(n_splits=3, random_state=42, shuffle=False)

# For each filter parameter
for i in range(len(fil)):
  f = 1

  # For each fold, train
  for train_index, test_index in cv.split(X, Y):

    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

    # Wandb init
    # wandb.init(project=f'Text_Emotion_CNN_{fil[i]}', name=f'Fold')

    # Clear session
    tf.keras.backend.clear_session()

    # Move here
    opt = tfa.optimizers.AdamW(lr=1e-5, weight_decay=1e-6)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.1)

    # LSTM Model
    model = build_cnn(fil[i])
    
    # Training
    print(f"Training {fil[i]} filters, fold {f}...")
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, Y_test),
                        callbacks=[reduce_lr, WandbCallback(monitor="val_loss", verbose=0, mode="auto", save_weights_only=False, save_model=False)])
    
    f = f+1

  # wandb.finish()


accuracy,▆▄▃▆▄▅▆▂▆▆▄▄▃▅█▅▃▃▄▂▁▃▃▄▃▁▆▃▄
epoch,▁▁▁▂▂▂▃▃▃▃▃▄▄▄▅▅▅▅▅▆▆▆▇▇▇▇▇██
loss,██▇▇▇▇▆▆▆▅▅▅▄▄▄▃▄▃▃▃▃▂▂▂▂▂▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,█▇▇▇▆▆▆▅▅▅▄▄▄▄▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁
val_loss,██▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▁▁▁
accuracy,0.05617
best_epoch,28
best_val_loss,2.57207
epoch,28
loss,2.57398


Training 64 filters, fold 1...
Epoch 1/50
3334/3334 [==============================] - 24s 7ms/step - loss: 2.2084 - accuracy: 0.2180 - val_loss: 2.2288 - val_accuracy: 0.1852 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 23s 7ms/step - loss: 2.1770 - accuracy: 0.2231 - val_loss: 2.2463 - val_accuracy: 0.1852 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 22s 6ms/step - loss: 2.1656 - accuracy: 0.2261 - val_loss: 2.2307 - val_accuracy: 0.1852 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 21s 6ms/step - loss: 2.1635 - accuracy: 0.2310 - val_loss: 2.2129 - val_accuracy: 0.1807 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 23s 7ms/step - loss: 2.1536 - accuracy: 0.2330 - val_loss: 2.2490 - val_accuracy: 0.1910 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 21s 6ms/step - loss: 2.1467 - accuracy: 0.2402 - val_loss: 2.2352 - val_accuracy: 0.1968 - lr: 1.0000e-

accuracy,▁▁▁▁▂▂▂▃▄▅▅▅▆▆▆▇▇▇██████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,██▇▇▇▇▇▆▆▅▅▄▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂
lr,███████████████████▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▂▄▅▆▆▆▆▆▇▇▇█▇███████████████████████
val_loss,███▇█▇▆▆▅▅▄▄▃▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.43404
best_epoch,28
best_val_loss,1.98726
epoch,49
loss,1.7916


Training 64 filters, fold 2...
Epoch 1/50
3334/3334 [==============================] - 24s 7ms/step - loss: 2.2264 - accuracy: 0.2020 - val_loss: 2.1476 - val_accuracy: 0.2518 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 21s 6ms/step - loss: 2.1869 - accuracy: 0.2110 - val_loss: 2.1376 - val_accuracy: 0.2510 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 23s 7ms/step - loss: 2.1784 - accuracy: 0.2173 - val_loss: 2.1464 - val_accuracy: 0.2426 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 22s 6ms/step - loss: 2.1703 - accuracy: 0.2212 - val_loss: 2.1356 - val_accuracy: 0.2510 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 21s 6ms/step - loss: 2.1616 - accuracy: 0.2344 - val_loss: 2.1348 - val_accuracy: 0.2531 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 23s 7ms/step - loss: 2.1597 - accuracy: 0.2350 - val_loss: 2.1261 - val_accuracy: 0.2543 - lr: 1.0000e-

accuracy,▁▁▁▂▂▂▂▃▃▄▄▄▅▅▆▆▆▇▇▇▇▇██████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▇▇▇▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,███████████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▂▂▁▂▂▂▂▂▄▄▄▅▆▇▇▇████████████████████████
val_loss,████▇▇▇▆▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.44891
best_epoch,24
best_val_loss,1.96914
epoch,49
loss,1.72216


...

accuracy,▁▁▁▁▂▃▃▃▄▄▅▅▆▆▆▆▇▇▇█████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▇▆▆▆▅▅▅▄▄▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂
lr,████████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▄▅▄▄▄▄▅▆▇▆▇▇████████████████████████
val_loss,▇▇██▇▆▆▇▆▆▄▄▃▃▂▂▁▁▂▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂
accuracy,0.479
best_epoch,20
best_val_loss,2.00221
epoch,49
loss,1.70808


Training 256 filters, fold 2...
Epoch 1/50
3334/3334 [==============================] - 25s 7ms/step - loss: 2.2015 - accuracy: 0.2098 - val_loss: 2.1478 - val_accuracy: 0.2450 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 23s 7ms/step - loss: 2.1811 - accuracy: 0.2193 - val_loss: 2.1404 - val_accuracy: 0.2465 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 22s 7ms/step - loss: 2.1715 - accuracy: 0.2284 - val_loss: 2.1357 - val_accuracy: 0.2505 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 24s 7ms/step - loss: 2.1635 - accuracy: 0.2348 - val_loss: 2.1308 - val_accuracy: 0.2531 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 24s 7ms/step - loss: 2.1563 - accuracy: 0.2401 - val_loss: 2.1289 - val_accuracy: 0.2540 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 22s 7ms/step - loss: 2.1464 - accuracy: 0.2460 - val_loss: 2.1202 - val_accuracy: 0.2552 - lr: 1.0000e

accuracy,▁▁▂▂▂▂▃▃▅▅▅▆▆▆▇▇▇███████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▇▇▆▅▅▄▄▃▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂
lr,██████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▁▁▂▂▂▃▄▅▆▇▇▇▇██████████████████████████
val_loss,███▇▇▇▆▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂
accuracy,0.44152
best_epoch,18
best_val_loss,1.96165
epoch,49
loss,1.73938


Training 256 filters, fold 3...
Epoch 1/50
3334/3334 [==============================] - 24s 7ms/step - loss: 2.1259 - accuracy: 0.2476 - val_loss: 2.4012 - val_accuracy: 0.1087 - lr: 1.0000e-05
Epoch 2/50
3334/3334 [==============================] - 24s 7ms/step - loss: 2.1037 - accuracy: 0.2552 - val_loss: 2.4057 - val_accuracy: 0.1280 - lr: 1.0000e-05
Epoch 3/50
3334/3334 [==============================] - 22s 7ms/step - loss: 2.0979 - accuracy: 0.2587 - val_loss: 2.3620 - val_accuracy: 0.1610 - lr: 1.0000e-05
Epoch 4/50
3334/3334 [==============================] - 22s 7ms/step - loss: 2.0927 - accuracy: 0.2673 - val_loss: 2.3890 - val_accuracy: 0.1579 - lr: 1.0000e-05
Epoch 5/50
3334/3334 [==============================] - 22s 7ms/step - loss: 2.0814 - accuracy: 0.2741 - val_loss: 2.3712 - val_accuracy: 0.1847 - lr: 1.0000e-05
Epoch 6/50
3334/3334 [==============================] - 22s 7ms/step - loss: 2.0758 - accuracy: 0.2766 - val_loss: 2.3358 - val_accuracy: 0.1942 - lr: 1.0000e

accuracy,▁▁▁▂▂▃▃▄▄▅▅▅▆▆▆▇▇███████████████████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,███▇▇▇▇▆▆▅▅▄▄▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂
lr,██████████████████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃▃▅▄▅▅▆▆▆▆▆▇▇█████████████████████████
val_loss,██▇█▆▇▅▆▄▄▃▃▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.42453
best_epoch,18
best_val_loss,2.11477
epoch,49
loss,1.73623
